In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [3]:
torch.__version__

'1.5.0'

In [4]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train': 
    datasets.ImageFolder('smaller_dataset/train', data_transforms['train']),
    'validation': 
    datasets.ImageFolder('smaller_dataset/val', data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=32,
                                shuffle=True, num_workers=4),
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=False, num_workers=4)
}

FileNotFoundError: [Errno 2] No such file or directory: 'smaller_dataset/train'

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
model = models.resnet50(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [9]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.detach() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.float() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss.item(),
                                                        epoch_acc.item()))
    return model

In [10]:
model_trained = train_model(model, criterion, optimizer, num_epochs=10)

Epoch 1/10
----------
train loss: 0.3995, acc: 0.8371
validation loss: 0.3678, acc: 0.8520
Epoch 2/10
----------
train loss: 0.3701, acc: 0.8510
validation loss: 0.3554, acc: 0.8579
Epoch 3/10
----------
train loss: 0.3616, acc: 0.8521
validation loss: 0.3646, acc: 0.8539
Epoch 4/10
----------
train loss: 0.3534, acc: 0.8550
validation loss: 0.3726, acc: 0.8437
Epoch 5/10
----------
train loss: 0.3531, acc: 0.8554
validation loss: 0.3690, acc: 0.8585
Epoch 6/10
----------
train loss: 0.3476, acc: 0.8592
validation loss: 0.3448, acc: 0.8553
Epoch 7/10
----------
train loss: 0.3469, acc: 0.8577
validation loss: 0.3448, acc: 0.8616
Epoch 8/10
----------
train loss: 0.3465, acc: 0.8575
validation loss: 0.3524, acc: 0.8514
Epoch 9/10
----------
train loss: 0.3419, acc: 0.8599
validation loss: 0.3396, acc: 0.8593
Epoch 10/10
----------
train loss: 0.3390, acc: 0.8606
validation loss: 0.3356, acc: 0.8634
